In [1]:
from os import listdir
import re
import pymorphy2
from tqdm import tqdm
from bs4 import BeautifulSoup
import codecs
import bs4
from pymystem3 import Mystem
from collections import defaultdict

In [2]:
import numpy as np
import pandas as pd
import os

##  Лемматизация

Эти ячейки не выполнены, потому что я один раз сделала лемматизацию в кегле и скачала, и все время работала с пиклами

In [ ]:
dires = ['20170726',
 '20170707',
 '20170708',
 '20170704',
 '20170710',
 '20170717',
 '20170702',
 '20170711'
        ]

In [ ]:
# path = "/kaggle/input/text-relevance-competition-ir-1-ts-fall-2020/content/content/"
# path_short = "/kaggle/input/text-relevance-competition-ir-1-ts-fall-2020/"
# pathtemp = "/kaggle/working/"
path = ""
path_short = ""
pathtemp = ""

NDOCS = 38114
stem = Mystem()

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_my = '''которых которые твой которой которого сих ком свой твоя этими слишком нами всему будь саму чаще ваше сами наш затем еще самих наши ту каждое мочь весь этим наша своих оба который зато те этих вся ваш такая теми ею которая нередко
каждая также чему собой самими нем вами ими откуда такие тому та очень сама нему алло оно этому кому тобой таки твоё
каждые твои мой нею самим ваши ваша кем мои однако сразу свое ними всё неё тех хотя всем тобою тебе одной другие
эта о само эта буду поэтому самой моё своей такое всею будут своего кого свои мог нам особенно её самому наше кроме вообще вон
мною никто это'''
stop_words = stopwords.words('russian')
stop_words.extend(stop_my.split())
len(stop_words)

In [ ]:
docs_info = defaultdict(list)

In [ ]:
def lemmatizeDocs(urls, titles, texts): # разделитель между текстами br!
    number = titles.count("br!")
    
    titles_l = "".join(stem.lemmatize(titles)).strip()
    texts_l = "".join(stem.lemmatize(texts)).strip()
    
    for url, title, text in zip(urls, titles_l.split("br!"), texts_l.split("br!")):
        docs_info[url] = [title, text]

In [ ]:
for dir in dires:

    alltitles = ""
    alltexts = ""
    urls_list = []
    for filename in tqdm(listdir(path + dir)):
            
        with codecs.open(path + dir + '/' + filename, 'r', 'utf-8', errors='ignore') as file:
            url = file.readline().strip('\n')
            html = file.read()
        
        soup = BeautifulSoup(html)
        
        [s.extract() for s in soup(['script', 'style', 'title'])]
        text = soup.get_text('\n', True).lower()
        text = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(text)
        alltexts += ' '.join([e for e in text if e not in stop_words]) + " br! "
    
        soup = BeautifulSoup(html)
        title = ' '.join([ e.get_text() for e in soup.find_all('title') if e.get_text() not in stop_words ]).lower()
        title = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(title)
        alltitles += ' '.join(title) + " br! " 
        
        urls_list.append(url)
        
        if i%10==0: #10 текстов
            lemmatizeDocs(urls_list, alltitles, alltexts)
            urls_list = []
            alltitles = ""
            alltexts = ""
    
    lemmatizeDocs(urls_list, alltitles, alltexts)

In [ ]:
import pickle
some_file = open(pathtemp+"docs_info.pickle", "wb")
pickle.dump(docs_info, some_file)
some_file.close()

In [ ]:
def lemmatizeHeaders(urls, headers):
    number = headers.count("br!")
    
    headers_l = "".join(stem.lemmatize(headers)).strip()
    
    for url, head in zip(urls, headers_l.split("br!")):
        Headers[url] = head 

In [ ]:
Headers = defaultdict(str)
for dir in dires:

    allheaders = ""
    urls_list = []
    for filename in tqdm(listdir(path + dir)):

        with codecs.open(path + dir + '/' + filename, 'r', 'utf-8', errors='ignore') as file:
            url = file.readline().strip('\n')
            html = file.read()
        
        soup = BeautifulSoup(html)
        
        header = ' '.join([e.get_text() for e in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]).lower()
        header = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(header)
        allheaders += ' '.join(header) + " br! " 
        
        urls_list.append(url)

        if i%100==0:
            lemmatizeHeaders(urls_list, allheaders)
            urls_list = []
            allheaders = ""
    
    lemmatizeHeaders(urls_list, allheaders)

In [ ]:
import pickle
some_file = open(pathtemp+"headers.pickle", "wb")
pickle.dump(Headers, some_file)
some_file.close()

## Создание словарей

In [3]:
path = ""
path_short = ""
pathtemp = ""
NDOCS = 38114
morph = pymorphy2.MorphAnalyzer()
stem = Mystem()

In [4]:
import pickle
some_file = open("docs_info.pickle", "rb")
docs_info = pickle.load(some_file)
some_file.close()

In [5]:
some_file = open("headers.pickle", "rb")
headers = pickle.load(some_file)
some_file.close()

In [6]:
urls_order = defaultdict(str)
#url_order -> url - и этот номер и есть номер в доках
with open(path_short + "urls.numerate.txt", 'r') as file:
    for _, line in enumerate(tqdm(file)):
        line = line.strip("\n").split("\t")
        urls_order[int(line[0])] = line[1]

38114it [00:00, 483330.92it/s]


In [7]:
docs2range = defaultdict(list)
with open(path_short + "sample.technosphere.ir1.textrelevance.submission.txt", 'r') as file:
    l = file.readline()
    for _, line in enumerate(tqdm(file)):
        line = line.strip("\n").split(",")
        docs2range[int(line[0])].append(int(line[1]))

38772it [00:00, 287395.43it/s]


In [8]:
alldocsNumber = 38114
avgtitleLen = 0
avgtextLen = 0
avgheaderLen = 0
titlesDict = defaultdict(int)
textDict = defaultdict(int)
headersDict = defaultdict(int)

#8 пустых документов

In [9]:
for key in tqdm(docs_info):
    doc = docs_info[key]
    
    title = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(doc[0].lower())
    text = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(doc[1].lower())
    
    avgtitleLen += len(title)
    avgtextLen += len(text)
    
    met = set()
    for word in title:
        if word not in met:
            titlesDict[word] += 1
            met.add(word)
    
    met = set()
    for word in text:
        if word not in met:
            textDict[word] += 1
            met.add(word)

100%|██████████| 38114/38114 [06:55<00:00, 91.69it/s] 


In [10]:
avgheaderLen = 0
headersDict = defaultdict(int)
for key in tqdm(headers): 
    header = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(headers[key].lower())
    avgheaderLen += len(header)   
    met = set()
    for word in header:
        if word not in met:
            headersDict[word] += 1
            met.add(word)

100%|██████████| 38114/38114 [00:06<00:00, 6041.94it/s] 


In [11]:
avgtitleLen /= 38114
avgtextLen /= 38114
avgheaderLen /= 38114
avgtextLen, avgtitleLen, avgheaderLen

(8998.754945689248, 9.041979325182348, 154.22760665372303)

In [12]:
#pre-count idfs for every word in titledict
import numpy as np
IDFall = 0
negIDF = []
for word in tqdm(titlesDict):
    IDF = np.log(alldocsNumber - titlesDict[word] + 0.5) - np.log(titlesDict[word] + 0.5)
    IDFall += IDF
    if IDF<0:
#         IDF = eps
        negIDF.append(word)
    titlesDict[word] = IDF
eps = 0.1 * IDFall / len(titlesDict)
for word in negIDF:
    titlesDict[word] = eps

100%|██████████| 36204/36204 [00:00<00:00, 158333.05it/s]


In [13]:
#pre-count idfs for every word in textdict
import numpy as np
IDFall = 0
negIDF = []
for word in tqdm(textDict):
    IDF = np.log(alldocsNumber - textDict[word] + 0.5) - np.log(textDict[word] + 0.5) #this formula?
    IDFall += IDF
    if IDF < 0:
#         IDF = eps
        negIDF.append(word)
    textDict[word] = IDF
eps = 0.1 * IDFall / len(textDict)
for word in negIDF:
    textDict[word] = eps

100%|██████████| 3390040/3390040 [00:14<00:00, 236731.15it/s]


In [14]:
#pre-count idfs for every word in headersdict
import numpy as np
IDFall = 0
negIDF = []
for word in tqdm(headersDict):
    IDF = np.log(alldocsNumber - headersDict[word] + 0.5) - np.log(headersDict[word] + 0.5)
    IDFall += IDF
    if IDF<0:
#         IDF = eps
        negIDF.append(word)
    headersDict[word] = IDF
eps = 0.1 * IDFall / len(headersDict)
for word in negIDF:
    headersDict[word] = eps

100%|██████████| 146342/146342 [00:00<00:00, 238791.98it/s]


## Ранжирование

In [15]:
from scipy.spatial.distance import cosine

In [16]:
doc_titles_texts = []
for key in docs_info.keys():
    doc_titles_texts.append(docs_info[key][0].split())

In [17]:
# doc_texts_texts = []
# for key in docs_info.keys():
#     doc_texts_texts.append(docs_info[key][1].split())

In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(doc_titles_texts)]
model = Doc2Vec(documents, vector_size=32, min_count=1, workers=10, epochs=50)

In [19]:
def cosine_similarity_reranking(query_raw, ranked_list, k=5):
    
    query_raw_lemm = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(' '.join(stem.lemmatize(query_raw.lower())))
    query_vector = model.infer_vector(query_raw_lemm)
    
    k = len(ranked_list)
    
    for i in range(k):
        doc_ord, doc_score = ranked_list[i]
        doc = docs_info[urls_order[doc_ord]]  
        title_vector = model.infer_vector(doc[0].split()) #143 дока без тайтлов
        doc_score += 30. * ( 1 - cosine(query_vector, title_vector) )
        ranked_list[i] = tuple([doc_ord, doc_score])
        
    ranked_list[:k] = sorted(ranked_list[:k], key=lambda x: x[1], reverse=True)
    return ranked_list

In [20]:
k = 2.0
b = 0.75

In [21]:
def calc_score(queryline, doc, IDFdict):
    score = 0
    docTF = defaultdict(float)
    doc_s = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(doc.lower())
    doc_len = len(doc_s)
    query = re.compile(r'[A-Za-zА-Яа-я0-9]+').findall(' '.join(stem.lemmatize(queryline.lower())))
    
    for word in doc_s:
        docTF[word] += 1
    
    for word in query:
        score += IDFdict[word] * (docTF[word]*(k+1)) / (docTF[word]+k*(1-b+b*doc_len/avgtitleLen))
    return score


def BM25(query, docs_list):
    scores = defaultdict(int)
    for docOrd in docs_list:
        score = 0
        url = urls_order[docOrd]
        doc = docs_info[url]
        
        score += 3.0 * calc_score(query, doc[0], titlesDict)
        score += 1.2 * calc_score(query, doc[1], textDict)
        score += 1.0 * calc_score(query, headers[url], headersDict)
        scores[docOrd] = score
        
        order = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        
    return order

In [22]:
ans_csv = []

query_file = "queries.numerate_fixed_fixed.txt" #синонимы транслиты исправлены опечатки классы запросов
query_raw_file = "queries.numerate_fixed.txt" # исправлены только самые очевидные опечатки

with open(query_file, 'r') as file, open(query_raw_file, 'r') as raw_file:
    for (_,line), (_,line_raw) in tqdm(zip(enumerate(file), enumerate(raw_file))):
    
        split = line.strip().split('\t')
        split_raw = line_raw.strip().split('\t')[1]
        
        docs_list = docs2range[int(split[0])]
        ranked_list = BM25(split[1], docs_list)
        reranked_list = cosine_similarity_reranking(split_raw, ranked_list)
        id_list = [int(split[0])]*len(reranked_list)
        
        ans_csv.extend(list(zip(id_list, reranked_list)))

399it [09:34,  1.44s/it]


In [24]:
print(ans_csv[:10])

[(1, (28, 133.84481657602134)), (1, (78, 107.09481768517452)), (1, (74, 99.63557695251605)), (1, (53, 92.74555112745723)), (1, (7, 79.33425321263576)), (1, (66, 79.000205615944)), (1, (57, 73.9324903902513)), (1, (85, 72.02227326672266)), (1, (82, 69.22521949302424)), (1, (44, 67.00715222528342))]


In [25]:
ans_col = [a[1][0] for a in ans_csv]
ans_q = [a[0] for a in ans_csv]
ans = list(zip(ans_q, ans_col))

df = pd.DataFrame(ans, columns = ['QueryId', 'DocumentId'])
df.to_csv("submission21.csv", index=False)